In [1]:
#---------------------------------------------------------------#
#          Notes on Census CSV file
#---------------------------------------------------------------#
#U.S. Census population data by State/County from 2010-2016
    #VARIABLE list: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2016/nst-est2016-alldata.pdf
    #Data source: https://www2.census.gov/programs-surveys/popest/datasets/2010-2016/counties/totals/?C=D;O=A
    #Column_names = list(county_census_pop.columns)
    #2016 Population estimate: POPESTIMATE2016
        #'STATE' is an int code // 'COUNTY' is an int code // 'STNAME' is string // 'CTYNAME' 'is the county name as string

# Read the CSV file with all str columns as lowercase // Shape: rows=3193, columns=116 // filesize: 2.5MB
# county_census_pop = pd.read_csv('co-est2016-alldata.csv',\
#                                 encoding="ISO-8859-1").apply(lambda x: x.astype(str).str.lower())

In [3]:
import matplotlib.pyplot as plt
import requests as req
import pandas as pd
import zipcodes
import json

In [4]:
#---------------------------------------------------------------#
#          User inputs 5-digit Zipcode to generate lat / lng
#---------------------------------------------------------------#

target_zip = input("Which zipcode would you like to score? ")
y = True
while y ==True:
    if len(zipcodes.matching(target_zip)) == 0:
        print("That is not a valid zip code")
        target_zip = input("Please try a new zipcode ")
    else:
        y = False

lat = zipcodes.matching(target_zip)[0]["lat"]
lng = zipcodes.matching(target_zip)[0]["long"]


Which zipcode would you like to score? 22314


In [5]:
# Function requires a latitude and longitude value
# Call this function to generate census population data for 2010 - 2016
def cen_block_query(lat,lng):
    # Queries Census for county/State associated to Lat/Long
    # API Info (No Key Required):  https://www.fcc.gov/general/census-block-conversions-api
    cen_block_url = ('http://data.fcc.gov/api/block/find?format=json&latitude=%s&longitude=%s&showall=true' % (lat, lng))
    lat_lon_county = req.get(cen_block_url).json()
    county_name = lat_lon_county['County']['name']
    state_name = lat_lon_county['State']['name']
    county_census_pop = pd.read_csv('Resources/co-est2016-alldata.csv',\
                                encoding="ISO-8859-1").apply(lambda x: x.astype(str).str.lower())
    # Match County and State name to retrieve population information from 2010 through 2016
    for index, row in county_census_pop.iterrows():
        if str.lower(county_name) in row['CTYNAME'] and row['STNAME'] == str.lower(state_name):
            years = ['2010','2011','2012','2013','2014','2015','2016']
            pops = []
            pops.append(int(row['POPESTIMATE2010']))
            pops.append(int(row['POPESTIMATE2011']))
            pops.append(int(row['POPESTIMATE2012']))
            pops.append(int(row['POPESTIMATE2013']))
            pops.append(int(row['POPESTIMATE2014']))
            pops.append(int(row['POPESTIMATE2015']))
            pops.append(int(row['POPESTIMATE2016']))           
            pop_dict = {'Years': years, 'Population': pops}
            # Return a dataframe with population value for each year
            pop_est = pd.DataFrame(pop_dict)
        else:
            next    
    return pop_est, county_name, state_name

#---------------------------------------------------------------#
# call this function to present a line graph of population change
def census_plot(pop_est,county_name,state_name):
    pop_len = len(pop_est['Population'])
    _2010 = pop_est['Population'][1]
    _2016 = pop_est['Population'][pop_len -1]
    if _2010 < _2016:
        #
        diff_ = (round(((_2016 - _2010)/ _2016) * 100))
        pop_growth = ((_2016 - _2010)/ _2016)
        diff_str = "Note:\nIncrease of population by\n" + str(diff_) + "% from 2010 to 2016"
    elif _2010 > _2016:
        diff_ = (round(((_2010 - _2016)/ _2010) * 100))
        diff_str = "Note:\nDecrease of population by\n" + str(diff_) + "% from 2010 to 2016"
    else:
        diff_str = "Note:\nPopulation estimated as\nthe same from 2010 to 2016"
    ax = pop_est.plot(figsize = (8,6),color='blue', legend=False, marker = '*',markersize=15)
    ax.set_xticklabels(pop_est['Years'], fontsize=13, rotation=45)
    plt.grid()
    plt.figtext(0.91,0.45,diff_str,fontsize=12)
    plt.title("Census Population Estimates (%s County, %s)"%(county_name,state_name), fontsize = 14)
    plt.ylabel("Population", fontsize=14)
    plt.savefig("Population_Change_LineGraph.png")
    return pop_growth

#---------------------------------------------------------------#
# Call this function to capture a DF that includes yearly changes in population
def population_df_generator(pop_est):
    pop_len = len(pop_est['Population'])
    pop_diff = [0]
    pop_diff_prcnt = [0]
    for x in range(pop_len-1):
        diff = (pop_est['Population'][x+1] - pop_est['Population'][x])
        pop_diff.append(diff)
        diff_prcnt = round(((diff/ pop_est['Population'][x]) * 100),2)
        pop_diff_prcnt.append(diff_prcnt)
    census_pop_master_df = pop_est
    census_pop_master_df['Difference'] = pop_diff
    census_pop_master_df['Percent Change'] = pop_diff_prcnt
    return census_pop_master_df

In [6]:
pop_est, county_name, state_name = cen_block_query(lat,lng)
pop_diff = census_plot(pop_est,county_name,state_name)
#population_df_generator(pop_est)

In [7]:
pop_diff

0.071619279892176368